This is a digit recognition program using Tensorflow in Python
\n I will take it step-by-step: \n
1. Import libraries and dataset
2. Verify dataset is good
3. Build the model
4. Train the model
5. Test the model


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input

In [2]:
# Load the MNIST dataset
numbers = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = numbers.load_data()

In [3]:
#Normalize the data
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [4]:
# Define the model
model = Sequential([
    Input((28,28)),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(10, activation = 'softmax')
])
# Compile the model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


In [5]:
# Train the model
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8533 - loss: 0.4697
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9664 - loss: 0.1108
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9786 - loss: 0.0702
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9833 - loss: 0.0521
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9871 - loss: 0.0423
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9901 - loss: 0.0305
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9917 - loss: 0.0268
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9919 - loss: 0.0248
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9937 - loss: 0.0184
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9946 - loss: 0.0166
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9951 - loss: 0.0147
Epoch 12/20
1875/1875 ━━━━━━━━

In [6]:
# Test the model
test_loss, test_acc = model.evaluate(x_test, y_test)
# Print the results
print(f'Test Loss:{test_loss}, Test accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step - accuracy: 0.9701 - loss: 0.1662
Test Loss:0.14076530933380127, Test accuracy: 0.9747999906539917


In [7]:
# Save the model
model.save('mnistdigit_model.h5')

# Building the App

Even though the model has been trained and tested, I want to make an app to make use of this model to detect the digits live and better visualize the output from the softmax layer. All of this will be done using Tkinter and Pillow. Even though the code below is part of one program, I am going to import the required libraries here instead of top.

In [12]:
# Import necessary libraries for GUI
import tkinter as tk
from tkinter import Canvas, Button, Label
from PIL import Image, ImageDraw, ImageOps
import os

In [38]:
# Define the application as a class
class GUIApp:
    def __init__(self, master):
        self.master = master
        master.title("Digit Recognizer") # Give title to the tkinter window
        master.geometry("600x600")
        master.resizable(True, True) # Make the window resizable
        master.configure(bg="#0f0606") # Set the background color of the window
        # self.setup_model() # Setup the trained model to recognize the new input
        self.create_widgets() # Create Widgets on the app   
        # self.reset_canvas() # Reset the whole canvas

        ''' So, that is the basic setup for tkinter, we are missing the defined functions which will be defined below,
        however I wanted to add an important note here about what I am about to do now, I am going to be using Pillow
        to create a Pillow image which will basically act as invisible layer to extract the data from the user input,
        in this case, the handrawn digit, the reason why I am using both Tkinter and Pillow is that Tkinter will not
        give me the pixel data from the user input, but Pillow will, so Tkinter is mainly used here to visualize the
        input and provide the GUI, whereas Pillow is being used to extract the user input "invisibly" and feed the
        data to the machine learning model.'''

        self.image = Image.new('L',(280,280)) # Create the Pillow Image, it is grayscale 'L' , 280 x 280 pixels
        self.draw = ImageDraw.Draw(self.image) # Ready the image to be drawn on

        self.last_x, self.last_y = None, None # Store coordinated from last position of drawing lines
        self.drawing = False # Flag to know whether mouse is clicked or not

        self.predict_id = None 
        ''' This stores the ID number from the after() call of Tkinter, to check for scheduled
        predictions, and cancel the previous ones, this is necessary to reduce the number of prediction calls while
        also making the app live and responsive without wasting many resources. '''

    # Define the model setup
    def setup_model(self):
        self.model = keras.models.load_model('mnistdigit_model.h5')
        self.success_label = Label(self.master, text="Model Successfully Loaded", font=("Helvetica", 16), fg="green")

    def create_widgets(self):
        # This function creates the widgets for the GUI
            
        # This is the main frame for the whole program
        main_frame = tk.Frame(self.master, bg="#0f0606", bd=0, relief="flat", padx=30, pady=30, highlightbackground="#2f0000", highlightthickness=2)
        main_frame.pack(padx=20, pady=20, fill="both", expand=True)
        main_frame.grid_columnconfigure((0,1), weight=1)
        main_frame.grid_rowconfigure(0, weight=1)

        # This is the smaller canvas for the user input
        canvas_frame = tk.Frame(main_frame, bg= "#0f0606")
        canvas_frame.grid(row=0, column=0, padx=20, pady=20, sticky="nsew") 
        canvas_frame.grid_rowconfigure(0, weight=1)
        canvas_frame.grid_rowconfigure((0,1,3), weight=0)
        canvas_frame.grid_rowconfigure(2, weight=1)

        # This is the the instruction label
        instruction_label = Label(canvas_frame, text="Draw a digit (0-9) in the box below", font=("Helvetica", 16, "bold"), bg="#200b0b", fg="white")
        instruction_label.pack(pady=10)
        # This is the canvas where the user will draw the digit
        self.canvas = Canvas(canvas_frame, width=280, height=280, bg="black", cursor="crosshair", bd = 2, relief="ridge", highlightbackground="#200b0b", highlightthickness=2)
        self.canvas.pack(pady=10)

        # Adding drawing binding to the canvas the actual functions will be defined below
        self.canvas.bind("<Button-1>", self.start_draw)
        self.canvas.bind("<B1-Motion>", self.draw_line)
        self.canvas.bind("<ButtonRelease-1>", self.stop_draw)

        # The clear button to clear the canvas
        clear_button = Button(canvas_frame, text="Clear Canvas", command = self.reset_canvas, font=("Helvetica", 14), bg="#200b0b", fg="white", activebackground="#2f0000", activeforeground="white")
        clear_button.pack(pady=10)
        
        # The model status label will be place here, the status will be updated after the model is loaded
        # self.model_status_label.pack(pady=10)

        # Now definiing the result area where the prediction will be shown
        result_frame = tk.Frame(main_frame, bg="#0f0606")
        result_frame.grid(row=0, column=1, padx=20, pady=20, sticky="nsew") 
        result_frame.grid_rowconfigure((0,1), weight=0)
        result_frame.grid_rowconfigure(2, weight=1)
        result_frame.grid_columnconfigure(0, weight=1)

        # The result label to show the prediction text
        self.result_label = Label(result_frame, text="Prediction: ", font=("Helvetica", 16), bg="#0f0606", fg="white")
        self.result_label.pack(pady=10)

        # Shwowing the actual prediction number ? is the placeholder
        self.prediction_number = Label(result_frame, text="?", font=("Helvetica", 64, "bold"), bg="#0f0606", fg="white")
        self.prediction_number.pack(pady=10)

        # Prediction Matrix label
        prediction_matrix_label = Label(result_frame, text="Prediction Matrix:", font=("Helvetica", 12), bg="#0f0606", fg="white")
        prediction_matrix_label.pack(pady=10)

        # The actual prediction matrix to show the probabilities of each digit
        self.prediction_matrix = tk.Frame(result_frame, bg="#200b0b", bd = 1, relief = "groove") 
        self.prediction_matrix.pack(fill = "x", expand = True, padx=10, pady=10)
        
        # Create the matrix labels for each digit
        # Using a dictionary to store the labels for easy access
        self.probability_labels = {}

        # Using a loop to create labels for each digit
        for i in range(10):
            # Create a frame for each row in the prediction matrix
            row_frame = tk.Frame(self.prediction_matrix, bg="#200b0b")
            row_frame.pack(fill="x", pady = 2)
            # Create labels for the digit and its probability
            digit_label = Label(row_frame, text =f"Digit {i}:", font=("Helvetica", 12), bg="#200b0b", fg="white")
            digit_label.pack(side="left", padx=5)
            probability_label = Label(row_frame, text="0.00%", font=("Helvetica", 12), bg="#200b0b", fg="white")
            probability_label.pack(side="left", padx=5)
            # Store the probability label in the dictionary
            self.probability_labels[i] = {"frame" : row_frame, "label": probability_label}

    def start_draw(self):
        pass

    def stop_draw(self):
        pass

    def draw_line(self):
        pass

    def reset_canvas(self):
        pass

    def model_status_label(self):
        pass

In [39]:
window = tk.Tk()
app = GUIApp(window)
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\PythonInstall\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: GUIApp.start_draw() takes 1 positional argument but 2 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\PythonInstall\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: GUIApp.draw_line() takes 1 positional argument but 2 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\PythonInstall\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: GUIApp.draw_line() takes 1 positional argument but 2 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\PythonInstall\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^